In [2]:
import chromadb
client = chromadb.PersistentClient()

In [ ]:
client.delete_collection("answers")

In [2]:
answers = client.create_collection(
    name="answers"
)

In [7]:
import pandas as pd
from tqdm import tqdm

df = pd.read_excel("../resources/comfort_datasets.xlsx")
df.sample(5)

,user,answer,class
25639,다 내려놓고 싶어...,많이 지치셨군요...잠시 주변의 나무를 봐봐요. 무거운 나뭇잎들을 지니면서 힘들지만...,사연
27826,오늘은 쉬는 날이다 ๑ᴗ ๑,오늘 하루 푹 쉬어요,사연
6291,영화관 혼자 왔어,영화는 원래 혼자 보는게 제일 재밌어요,일상
14847,자려고 침대에 누워도 잠이 안와요.,하루 동안 열심히 살아왔으니 이제 푹 쉬세요. 내일 아침에는 개운하게 일어날 수 있...,증상
31334,올 여름휴가 때 여럿이 함께 하려면 어떻게 해야 하지?,부모님이나 친구들과 함께 휴가 계획을 짜 보는 건 어떨까요? 봉사활동이나 동아리 가...,사연


In [8]:
len(df)

35363

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/opt/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
ids = []
metadatas = []
embeddings = model.encode(df['user'],
                          normalize_embeddings=True,
                          show_progress_bar=True)

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1].user
    answer = row[1].answer
    
    metadata = {
        "query": query,
        "answer": answer
    }
    ids.append(str(index))
    metadatas.append(metadata)
    
chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings)]  

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size
    
    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    
    # chunk를 answers에 추가
    answers.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

Batches: 100%|████████████████████████████████| 397/397 [01:23<00:00,  4.76it/s]
12701it [00:00, 33037.14it/s]
100%|███████████████████████████████████████████| 13/13 [00:08<00:00,  1.62it/s]


In [1]:
result = answers.query(
    query_embeddings=model.encode("자꾸만 눈물이 나와", normalize_embeddings=True).tolist(),
    n_results=1
)

result_df = pd.DataFrame(result['metadatas'][0])
result_df['distances'] = result['distances'][0]
result_df

NameError: name 'answers' is not defined

In [38]:
result['metadatas'][0][0]['answer']

'왜 울어요? 무슨 일 있어요?'

In [3]:
len(df)

NameError: name 'df' is not defined